In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import nyaset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.utils.data as data
import numpy as np
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import sys,os
sys.path.append(os.pardir)
import copy

In [2]:
x_train,y_train = nyaset.load()

succesfully loaded NyasetData/0000000.txt
succesfully loaded NyasetData/0000001.txt
succesfully loaded NyasetData/0000002.txt
succesfully loaded NyasetData/0000003.txt
succesfully loaded NyasetData/0000004.txt
succesfully loaded NyasetData/0000005.txt
succesfully loaded NyasetData/0000006.txt
succesfully loaded NyasetData/0000007.txt
succesfully loaded NyasetData/0000008.txt
succesfully loaded NyasetData/0000009.txt
succesfully loaded NyasetData/0000010.txt
succesfully loaded NyasetData/0000011.txt
succesfully loaded NyasetData/0000012.txt
succesfully loaded NyasetData/0000013.txt
succesfully loaded NyasetData/0000014.txt
succesfully loaded NyasetData/0000015.txt
succesfully loaded NyasetData/0000016.txt
succesfully loaded NyasetData/0000017.txt
succesfully loaded NyasetData/0000018.txt
succesfully loaded NyasetData/0000019.txt


In [3]:
y_train = y_train.reshape(-1,64)

In [4]:
x_train = torch.tensor(x_train,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)

In [5]:
print(x_train.shape)
print(y_train.shape)

torch.Size([1199046, 2, 8, 8])
torch.Size([1199046, 64])


In [6]:
device = 'cpu'

In [7]:
class Bias(nn.Module):
    def __init__(self, input_shape):
        super(Bias, self).__init__()
        self.W = nn.Parameter(torch.zeros(input_shape[1:]))

    def forward(self, inputs):
        return inputs + self.W

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(2, 12, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(12, 50, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(50, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 50, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(50, 12, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(12, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(8, 1, kernel_size=1, bias=True),
        )
        self.flatten = nn.Flatten()
        self.bias = Bias((1, 64))
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.bias(x)
        x = self.softmax(x)
        return x

In [8]:
model = SimpleCNN()
# Assuming input has shape (batch_size, 2, 8, 8)
input_tensor = torch.randn(64,2, 8, 8)
output = model(input_tensor)
print(output)

tensor([[0.0157, 0.0156, 0.0156,  ..., 0.0156, 0.0157, 0.0157],
        [0.0157, 0.0156, 0.0156,  ..., 0.0156, 0.0157, 0.0157],
        [0.0157, 0.0156, 0.0156,  ..., 0.0156, 0.0157, 0.0157],
        ...,
        [0.0157, 0.0156, 0.0156,  ..., 0.0157, 0.0157, 0.0157],
        [0.0157, 0.0156, 0.0156,  ..., 0.0157, 0.0157, 0.0157],
        [0.0157, 0.0156, 0.0156,  ..., 0.0157, 0.0157, 0.0157]],
       grad_fn=<SoftmaxBackward0>)


In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

batch_size = 64
rng = np.random.default_rng()

In [10]:
def train(model, loss_fn, optimizer,x_train,y_train):
    train_size = len(x_train)
    for i in range(train_size//batch_size):
        indices = rng.integers(train_size, size=batch_size)
        X, y = x_train[indices], y_train[indices]
        X, y = X.to(device), y.to(device)
        
        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % batch_size == 0:
            loss, current = loss.item(), i * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

In [11]:
x_train[1]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 1., 1., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0.]]])

In [ ]:
train(model,loss_fn,optimizer,x_train,y_train)

loss: 4.158885  [    0/1199046]
loss: 4.158882  [ 4096/1199046]
loss: 4.158883  [ 8192/1199046]
loss: 4.158880  [12288/1199046]
loss: 4.158884  [16384/1199046]
loss: 4.158883  [20480/1199046]
loss: 4.158882  [24576/1199046]
loss: 4.158880  [28672/1199046]
loss: 4.158879  [32768/1199046]
loss: 4.158882  [36864/1199046]
loss: 4.158884  [40960/1199046]
loss: 4.158884  [45056/1199046]
loss: 4.158881  [49152/1199046]
loss: 4.158883  [53248/1199046]
loss: 4.158883  [57344/1199046]
loss: 4.158881  [61440/1199046]
loss: 4.158880  [65536/1199046]
loss: 4.158882  [69632/1199046]
loss: 4.158881  [73728/1199046]
loss: 4.158881  [77824/1199046]
loss: 4.158881  [81920/1199046]
loss: 4.158885  [86016/1199046]
loss: 4.158885  [90112/1199046]
loss: 4.158883  [94208/1199046]
loss: 4.158882  [98304/1199046]
loss: 4.158883  [102400/1199046]
loss: 4.158885  [106496/1199046]
loss: 4.158880  [110592/1199046]
loss: 4.158884  [114688/1199046]
loss: 4.158885  [118784/1199046]
loss: 4.158879  [122880/1199046]
lo